In [1]:
import findspark
findspark.init('/home/nardao666/spark-2.4.0-bin-hadoop2.7/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('Ecommerce-Customers.csv', inferSchema = True, header = True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
for item in data.head(2)[1]:
    print (item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
assembler = VectorAssembler(inputCols = ['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'], outputCol= 'features')

In [13]:
output = assembler.transform(data)

In [14]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [15]:
final_data = output.select('features','Yearly Amount Spent')

In [16]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [17]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [18]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                356|
|   mean|  497.1755767540653|
| stddev|  79.76131151082015|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [19]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [20]:
lr_model = lr.fit(train_data)

In [21]:
test_results = lr_model.evaluate(test_data)

In [22]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  5.989711325182498|
|-13.663400170902094|
|-1.3111902436190803|
| -4.823870260589786|
|  -6.13901154244661|
| -9.480623481616647|
| 3.3138704436869943|
|  -18.1955969511655|
| -14.88727119559212|
|-2.5358137531731018|
|  17.57350422594527|
|  6.219110935028482|
|-1.4836153643246917|
| 1.7078840048935149|
|-18.047179799547223|
|-2.2481794455578665|
|-17.649559815380428|
|  5.029852838120291|
|  -4.77127764300883|
|  16.99564983444361|
+-------------------+
only showing top 20 rows



In [23]:
test_results.rootMeanSquaredError

9.728436956190743

In [24]:
test_results.r2

0.9844247462932184

In [25]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [26]:
unlabel_data = test_data.select('features')

In [27]:
unlabel_data.show()

+--------------------+
|            features|
+--------------------+
|[30.9716756438877...|
|[31.0662181616375...|
|[31.2606468698795...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.5316044825729...|
|[31.5702008293202...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6548096756927...|
|[31.6610498227460...|
|[31.7366356860502...|
|[31.8164283341993...|
|[31.8627411090001...|
|[31.9048571310136...|
|[31.9480174211613...|
|[31.9673209478824...|
|[32.0180740106320...|
+--------------------+
only showing top 20 rows



In [28]:
predictions = lr_model.transform(unlabel_data)

In [29]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.9716756438877...| 488.6488984317102|
|[31.0662181616375...|462.59669337857645|
|[31.2606468698795...|422.63782150057045|
|[31.5171218025062...| 280.7422909109755|
|[31.5257524169682...| 450.1046383523285|
|[31.5261978982398...|418.57514967395446|
|[31.5316044825729...|433.20173528567557|
|[31.5702008293202...| 564.1410890925704|
|[31.5741380228732...|  559.296543356179|
|[31.5761319713222...| 543.7623977425014|
|[31.6005122003032...|461.59934726515166|
|[31.6548096756927...|469.04431279252003|
|[31.6610498227460...| 417.8419689442255|
|[31.7366356860502...| 495.2255622506384|
|[31.8164283341993...| 519.1696713032036|
|[31.8627411090001...| 558.5463206196046|
|[31.9048571310136...|491.59941723819657|
|[31.9480174211613...| 456.8910240547775|
|[31.9673209478824...| 450.5211188826611|
|[32.0180740106320...| 340.7874609108717|
+--------------------+------------